In [ ]:
import pandas as pd
test_play1 = pd.DataFrame({'col1': [1, 2, 3], 'col2': ['a', 'b', 'c']})
test_play2 = pd.DataFrame({'col1': [4, 5, 6], 'col2': ['d', 'e', 'f']})
test_dataset = [test_play1,test_play2]

In [ ]:
import torch
from torch_geometric.data import Data

for play in test_dataset:
    # Set play variables to starting values
    play_time_start = None
    play_data = []

    for i in play.index:
        # Read each data from each action
        node_data = []
        action_data = []

        # Start by reading current time
        raw_time_string = play["timestamp"]
        total = 0
        for part in raw_time_string.split():
            if part.endswith('µs'):  total += float(part[:-2]) / 1_000_000
            elif part.endswith('ms'): total += float(part[:-2]) / 1000
            elif part.endswith('m'):  total += float(part[:-1]) * 60
            elif part.endswith('s'):  total += float(part[:-1])
        play_time_current = float(round((total),2))

        # Check if we have starting time, set scaled starting time.
        if play_time_start is None:
            play_time_start = play_time_current
            play_time_current_scaled = play_time_start
        else:
            play_time_current_scaled = play_time_start - play_time_current

        # Start building action data
        node_data.append(play.iloc[i]["player_id"])
        action_data.append(play.iloc[i]["event_type"])
        # Balls coords and distance
        x1, y1 = play.iloc[i]["coordinates_x"], play.iloc[i]["coordinates_y"]
        x2, y2 = play.iloc[i]["end_coordinates_x"], play.iloc[i]["end_coordinates_y"]
        action_data.extend([x1, y1, x2, y2, ((x2-x1)**2 + (y2-y1)**2)**0.5])
        # 0 if null or false, 1 if true
        action_data.append(int(play.iloc[i]["is_under_pressure"] == "true"))

    play_data.append(action_data)
        